In [8]:
from collections import defaultdict
from typing import Any, List, Set, Tuple

import dill
import numpy as np

from Settings import Settings
from crel_helper import get_cr_tags
from results_procesor import is_a_regular_code
from load_data import load_process_essays
from window_based_tagger_config import get_config
from wordtagginghelper import merge_dictionaries
from BrattEssay import ANAPHORA

DATASET = "CoralBleaching" # SkinCancer

settings = Settings()
root_folder = settings.data_directory + DATASET + "/Thesis_Dataset/"
training_folder = root_folder + "Training" + "/"
test_folder = root_folder + "Test" + "/"
# NOTE: These predictions are generated from the "./notebooks/SEARN/Keras - Train Tagger and Save CV Predictions For Word Tags.ipynb" notebook
# used as inputs to parsing model
coref_root = root_folder + "CoReference/"
coref_output_folder = coref_root + "CRel/"

training_folder, test_folder

Results Dir: /Users/simon.hughes/Google Drive/Phd/Results/
Data Dir:    /Users/simon.hughes/Google Drive/Phd/Data/
Root Dir:    /Users/simon.hughes/GitHub/NlpResearch/
Public Data: /Users/simon.hughes/GitHub/NlpResearch/Data/PublicDatasets/


('/Users/simon.hughes/Google Drive/Phd/Data/CoralBleaching/Thesis_Dataset/Training/',
 '/Users/simon.hughes/Google Drive/Phd/Data/CoralBleaching/Thesis_Dataset/Test/')

In [6]:
train_fname = coref_output_folder + "training_crel_anatagged_essays.dill"
with open(train_fname, "rb") as f:
    pred_tagged_essays_train = dill.load(f)

test_fname = coref_output_folder + "test_crel_anatagged_essays.dill"
with open(test_fname, "rb") as f:
    pred_tagged_essays_test = dill.load(f)
    
print(len(pred_tagged_essays_train), len(pred_tagged_essays_test))

902 226


In [9]:
all_essays = pred_tagged_essays_train + pred_tagged_essays_test
len(all_essays)

1128

In [42]:
num_words = 0
words_with_anaphora_tag = 0
words_with_cc = 0
cc_with_anaphora_tag = 0

for e in all_essays:
    for sent in e.sentences:
        for wd, tags in sent:
            num_words +=1
            has_cc_code = False
            for t in tags:
                if is_a_regular_code(t):
                    has_cc_code = True
                    break

            if ANAPHORA in tags:
                words_with_anaphora_tag +=1
            
            if has_cc_code:
                words_with_cc += 1
                
            if ANAPHORA in tags and has_cc_code:
                cc_with_anaphora_tag +=1                
                    
#                     reg_tally[t] += 1
#                 if ANAPHORA in t and "other" not in t:
#                     if "->" in t:
#                         cr_tally[t] += 1
#                     elif "Anaphor:[" in t:
#                         cc_tally[t] += 1

# reg_tags = sorted(reg_tally.keys())
# all_ana_tags = sorted(cc_tally.keys())
# assert len(reg_tags) == len(all_ana_tags)
# all_ana_tags

num_words, words_with_anaphora_tag, cc_with_anaphora_tag, words_with_cc

(167865, 377, 36, 40041)

In [43]:
print("Words with Anaphora tags:         {wds_ana_cnt},\t - {wds_ana:.4f}%".format(
    wds_ana_cnt=words_with_anaphora_tag, 
    wds_ana=100* words_with_anaphora_tag/num_words))
print("Concept codes with Anaphora tags: {cc_ana},\t - {cc_ana_pct:.4f} % of cc codes, \t{wds_ana_cc_pct:.4f} % of ana codes".format(
    cc_ana=cc_with_anaphora_tag,
    cc_ana_pct=100* cc_with_anaphora_tag/words_with_cc,
    wds_ana_cc_pct=100* cc_with_anaphora_tag/words_with_anaphora_tag))

Words with Anaphora tags:         377,	 - 0.2246%
Concept codes with Anaphora tags: 36,	 - 0.0899 % of cc codes, 	9.5491 % of ana codes
